In [17]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier,RidgeClassifier,PassiveAggressiveClassifier
import pickle
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from datetime import datetime
import matplotlib.pyplot as plt
from pandas_ml import ConfusionMatrix 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix 
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import collections as ct
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords

In [96]:
input_df = pd.read_csv('incident_1month_resolved.csv')
# input_df = input_df[(input_df["Source"] == "Email")][["Subject","Symptom"]]
# input_df = input_df.reset_index()
print(input_df.shape)

(11304, 3)


In [6]:
# input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\r\n\\s*)+',value=' \r\n ')
# input_df['Symptom'] = input_df['Symptom'].replace(regex=["\\sFrom:.*\\s",
#                         "\\sTo:.*\\s","\\sDate:.*\\s","\\sSubject:.*\\s","\\sSent:.*\\s",
#                         "\\sCc:.*\\s","\\sBcc:\\s"], value='')
# input_df['Symptom'] = input_df['Symptom'].replace(regex=[                                                         
#                         "(?s)\\sThis message is intended .* destroy all copies\\.?\\s",
#                         "(?s)\\sDisclaimer .* only if necessary\\.?\\s",
#                         "(?s)\\sConfidentiality Note:.*original message\\.?\\s"], value='')
# input_df['Symptom'] = input_df['Symptom'].replace(regex='\\s?\\(AutoClosed\\)', value = '')

In [156]:
a = input_df['Symptom'][2]
print(a)
print("\n------\n")
# from nltk import word_tokenize, pos_tag, ne_chunk
# tagged = ne_chunk(pos_tag(word_tokenize(a)))
# person = []
# for subtree in tagged.subtrees(filter=lambda t: t.label() == 'PERSON'):
#         for leaf in subtree.leaves():
#             person.append(leaf[0])
            
# print(person)
print("\n------\n")
from nltk.corpus import stopwords
stop = stopwords.words('english')

print(stop)

Hello,
 
 
 
IPCM ITSM down system pager number 719-278-7264 not routing to on call pager.
 
 
 
Regards,
 
 
 
    

         
Glenn Secrest
 
Technical Analyst 2, 

Global Support Services
 
     
   
o +1 719 278.7273
 
w  www.ivanti.com
   
   
 
 
    

 
     
   
Note: My new email address is: Glenn.Secrest@ivanti.com
   
   
 
 
8415 Explorer Drive, Suite 150 Colorado Springs, CO 80920
 
 
 
This message is intended for the addressee(s) only and may contain confidential or privileged information. 

Any use of this information by persons other than addressee(s) is prohibited. 

If you have received this message in error, please reply to the sender and delete or destroy all copies. (AutoClosed)

------


------

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', '

In [282]:
special_chars = string.punctuation  
stop = stopwords.words('english')
from datetime import datetime
import math

def junk_clean(text):
    
    starttime1 = datetime.now()
    junk = []
    body = []
    b = text.splitlines()
    for index, i in enumerate(b):
        c = i.split()
        if len(c) > 0:
            sp = sum(v for k, v in ct.Counter(i).items() if k in special_chars)
            #name_perc = len(list(set(c).intersection(person)))/len(c)
            stop_perc = len(list(set(c).intersection(stop)))
            score = (stop_perc - math.log(sp)) if sp != 0 else float('Inf')
            sp_score = len(c)*sp
            print(index, "\t", len(c), "\t","%.2f" % sp ,"\t","%.2f" % stop_perc, "\t", "%.2f" % score, "\t" + i)
            if ((score <= 1) or (score == float('Inf'))):
                junk.append(i)       
            else:
                body.append(i)
        else:
            continue
            
    #print("Person Tagging: %s" % (datetime.now() - starttime1))
    
    junk = '\n'.join(junk)
    body = '\n'.join(body)
    return pd.Series({'junk': junk, 'body': body})

a = junk_clean(input_df["Symptom"][45])
# print("email:\n",input_df["Symptom"][675])
print("\n-----\n")
print("body:\n",a["body"])
print("\n-----\n")
print("junk:\n",a["junk"])

#39,41,

0 	 2 	 1.00 	 0.00 	 0.00 	Dear Rebecca, 
4 	 48 	 6.00 	 15.00 	 13.21 	thanks for the attached, which showed another delay from the barracuda. In order to permanently fix this issues, and exclude your IP’s from the rate control setting, please provide us a full list of sending IP addresses, that we then will be adding to the safe sender list. 
6 	 15 	 2.00 	 5.00 	 4.31 	Troy’s last fix is still in place, but seems like you sent from another address. 
8 	 1 	 0.00 	 0.00 	 inf 	Thanks
10 	 1 	 0.00 	 0.00 	 inf 	Ralph
14 	 3 	 0.00 	 0.00 	 inf 	Mit freundlichen Grüssen
16 	 2 	 0.00 	 0.00 	 inf 	Kind regards
20 	 2 	 0.00 	 0.00 	 inf 	Ralph Schöll
22 	 3 	 1.00 	 0.00 	 0.00 	Manager, Infrastructure Systems
24 	 4 	 2.00 	 0.00 	 -0.69 	VMware Certified Professional 4/5/Cloud
25 	 5 	 0.00 	 0.00 	 inf 	 NetApp Certified Data Management Administrator
27 	 5 	 0.00 	 0.00 	 inf 	HEAT DSM 7 certified engineer
31 	 2 	 0.00 	 0.00 	 inf 	HEAT Software
33 	 10 	 5.00 	 0.00 	 -1.61 

In [27]:
special_chars = string.punctuation  
stop = stopwords.words('english')
from datetime import datetime

def junk_clean(text):
    
#     starttime = datetime.now()
#     tagged = ne_chunk(pos_tag(word_tokenize(text)))
#     person = []
#     for subtree in tagged.subtrees(filter=lambda t: t.label() == 'PERSON'):
#         for leaf in subtree.leaves():
#             person.append(leaf[0])
#     print("Person Tagging: " + (datetime.now() - starttime))
    
    starttime1 = datetime.now()
    junk = []
    body = []
    b = text.splitlines()
    for index, i in enumerate(b):
        c = i.split()
        if len(c) > 0:
            sp = sum(v for k, v in ct.Counter(i).items() if k in special_chars)/len(c)
            #name_perc = len(list(set(c).intersection(person)))/len(c)
            stop_perc = len(list(set(c).intersection(stop)))/len(c)
            if len(c) < 10  or sp >= 0.5 or stop_perc <= 0.05:
                junk.append(i)
                #print(index, "\t", "%.2f" % name_perc, "\t",len(c),"\t","%.2f" % sp ,"\t","%.2f" % stop_perc, "\t", i)
            else:
                body.append(i)
        else:
            continue
            
    #print("Person Tagging: %s" % (datetime.now() - starttime1))
    
    junk = '\n'.join(junk)
    body = '\n'.join(body)
    return pd.Series({'junk': junk, 'body': body})

In [37]:
a = junk_clean(input_df["Symptom"][675])
print("email:\n",input_df["Symptom"][675])
print("\n-----\n")
print("body:\n",a["body"])
print("\n-----\n")
print("junk:\n",a["junk"])

email:
 Hi,
 
 
 
I am trying to install VMware Workstation on my machine. I downloaded it but I think it requires administrative privileges which I do not have. Can you help me with the installation?
 
 
 
Thanks,
 
Pam. (AutoClosed)

-----

body:
 I am trying to install VMware Workstation on my machine. I downloaded it but I think it requires administrative privileges which I do not have. Can you help me with the installation?

-----

junk:
 Hi,
Thanks,
Pam. (AutoClosed)


In [8]:
new = input_df["Symptom"].apply(junk_clean)

Person Tagging: 0:00:00.000224
Person Tagging: 0:00:00.000045
Person Tagging: 0:00:00.000156
Person Tagging: 0:00:00.000307
Person Tagging: 0:00:00.000055
Person Tagging: 0:00:00.000950
Person Tagging: 0:00:00.000049
Person Tagging: 0:00:00.000122
Person Tagging: 0:00:00.000060
Person Tagging: 0:00:00.000100
Person Tagging: 0:00:00.000260
Person Tagging: 0:00:00.000095
Person Tagging: 0:00:00.000052
Person Tagging: 0:00:00.000027
Person Tagging: 0:00:00.000032
Person Tagging: 0:00:00.000123
Person Tagging: 0:00:00.000027
Person Tagging: 0:00:00.000326
Person Tagging: 0:00:00.000450
Person Tagging: 0:00:00.000071
Person Tagging: 0:00:00.000083
Person Tagging: 0:00:00.000085
Person Tagging: 0:00:00.000097
Person Tagging: 0:00:00.000157
Person Tagging: 0:00:00.000781
Person Tagging: 0:00:00.000583
Person Tagging: 0:00:00.002524
Person Tagging: 0:00:00.000087
Person Tagging: 0:00:00.000083
Person Tagging: 0:00:00.000081
Person Tagging: 0:00:00.000205
Person Tagging: 0:00:00.000102
Person T

Person Tagging: 0:00:00.000430
Person Tagging: 0:00:00.000036
Person Tagging: 0:00:00.000032
Person Tagging: 0:00:00.000089
Person Tagging: 0:00:00.000132
Person Tagging: 0:00:00.000035
Person Tagging: 0:00:00.000077
Person Tagging: 0:00:00.000151
Person Tagging: 0:00:00.000069
Person Tagging: 0:00:00.000179
Person Tagging: 0:00:00.001079
Person Tagging: 0:00:00.001062
Person Tagging: 0:00:00.000902
Person Tagging: 0:00:00.000217
Person Tagging: 0:00:00.000753
Person Tagging: 0:00:00.000083
Person Tagging: 0:00:00.000166
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000267
Person Tagging: 0:00:00.000177
Person Tagging: 0:00:00.000053
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000219
Person Tagging: 0:00:00.000033
Person Tagging: 0:00:00.000146
Person Tagging: 0:00:00.000033
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.001103
Person Tagging: 0:00:00.000127
Person Tagging: 0:00:00.000153
Person Tagging: 0:00:00.000105
Person Tagging: 0:00:00.000096
Person T

Person Tagging: 0:00:00.000069
Person Tagging: 0:00:00.000076
Person Tagging: 0:00:00.000145
Person Tagging: 0:00:00.000033
Person Tagging: 0:00:00.000097
Person Tagging: 0:00:00.000187
Person Tagging: 0:00:00.000026
Person Tagging: 0:00:00.000173
Person Tagging: 0:00:00.000038
Person Tagging: 0:00:00.000035
Person Tagging: 0:00:00.000034
Person Tagging: 0:00:00.000204
Person Tagging: 0:00:00.000146
Person Tagging: 0:00:00.000035
Person Tagging: 0:00:00.000124
Person Tagging: 0:00:00.000267
Person Tagging: 0:00:00.000662
Person Tagging: 0:00:00.000044
Person Tagging: 0:00:00.000070
Person Tagging: 0:00:00.000138
Person Tagging: 0:00:00.000053
Person Tagging: 0:00:00.000082
Person Tagging: 0:00:00.000057
Person Tagging: 0:00:00.000221
Person Tagging: 0:00:00.000098
Person Tagging: 0:00:00.000146
Person Tagging: 0:00:00.000191
Person Tagging: 0:00:00.000147
Person Tagging: 0:00:00.000355
Person Tagging: 0:00:00.000132
Person Tagging: 0:00:00.000112
Person Tagging: 0:00:00.000072
Person T

Person Tagging: 0:00:00.000097
Person Tagging: 0:00:00.000072
Person Tagging: 0:00:00.000360
Person Tagging: 0:00:00.000038
Person Tagging: 0:00:00.000822
Person Tagging: 0:00:00.000150
Person Tagging: 0:00:00.000132
Person Tagging: 0:00:00.000110
Person Tagging: 0:00:00.000144
Person Tagging: 0:00:00.000053
Person Tagging: 0:00:00.000194
Person Tagging: 0:00:00.000323
Person Tagging: 0:00:00.000076
Person Tagging: 0:00:00.000200
Person Tagging: 0:00:00.001471
Person Tagging: 0:00:00.000864
Person Tagging: 0:00:00.000234
Person Tagging: 0:00:00.000348
Person Tagging: 0:00:00.000028
Person Tagging: 0:00:00.000176
Person Tagging: 0:00:00.000508
Person Tagging: 0:00:00.000496
Person Tagging: 0:00:00.000077
Person Tagging: 0:00:00.000175
Person Tagging: 0:00:00.000029
Person Tagging: 0:00:00.000163
Person Tagging: 0:00:00.000238
Person Tagging: 0:00:00.000120
Person Tagging: 0:00:00.000201
Person Tagging: 0:00:00.000040
Person Tagging: 0:00:00.000073
Person Tagging: 0:00:00.000068
Person T

Person Tagging: 0:00:00.000300
Person Tagging: 0:00:00.000303
Person Tagging: 0:00:00.000084
Person Tagging: 0:00:00.000033
Person Tagging: 0:00:00.000097
Person Tagging: 0:00:00.000080
Person Tagging: 0:00:00.000032
Person Tagging: 0:00:00.000377
Person Tagging: 0:00:00.000033
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000038
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000032
Person Tagging: 0:00:00.000030
Person Tagging: 0:00:00.000031
Person Tagging: 0:00:00.000034
Person Tagging: 0:00:00.000043
Person Tagging: 0:00:00.000035
Person Tagging: 0:00:00.000094
Person Tagging: 0:00:00.000264
Person Tagging: 0:00:00.000046
Person Tagging: 0:00:00.000050
Person Tagging: 0:00:00.000071
Person Tagging: 0:00:00.000071
Person Tagging: 0:00:00.000072
Person Tagging: 0:00:00.000034
Person Tagging: 0:00:00.000885
Person Tagging: 0:00:00.000171
Person Tagging: 0:00:00.000304
Person Tagging: 0:00:00.000157
Person T

...

Person Tagging: 0:00:00.000038
Person Tagging: 0:00:00.000123
Person Tagging: 0:00:00.000194
Person Tagging: 0:00:00.000199
Person Tagging: 0:00:00.000083
Person Tagging: 0:00:00.000188
Person Tagging: 0:00:00.000067
Person Tagging: 0:00:00.000419
Person Tagging: 0:00:00.000463
Person Tagging: 0:00:00.000114
Person Tagging: 0:00:00.000270
Person Tagging: 0:00:00.000154
Person Tagging: 0:00:00.000143
Person Tagging: 0:00:00.000270
Person Tagging: 0:00:00.000289
Person Tagging: 0:00:00.000168
Person Tagging: 0:00:00.000213
Person Tagging: 0:00:00.000151
Person Tagging: 0:00:00.000142
Person Tagging: 0:00:00.000071
Person Tagging: 0:00:00.000166
Person Tagging: 0:00:00.000119
Person Tagging: 0:00:00.000338
Person Tagging: 0:00:00.000048
Person Tagging: 0:00:00.000032
Person Tagging: 0:00:00.000170
Person Tagging: 0:00:00.000648
Person Tagging: 0:00:00.000129
Person Tagging: 0:00:00.000144
Person Tagging: 0:00:00.000033
Person Tagging: 0:00:00.000612
Person Tagging: 0:00:00.000147
Person T

Person Tagging: 0:00:00.000163
Person Tagging: 0:00:00.000061
Person Tagging: 0:00:00.000214
Person Tagging: 0:00:00.000043
Person Tagging: 0:00:00.000215
Person Tagging: 0:00:00.000074
Person Tagging: 0:00:00.000269
Person Tagging: 0:00:00.000369
Person Tagging: 0:00:00.000088
Person Tagging: 0:00:00.000087
Person Tagging: 0:00:00.000328
Person Tagging: 0:00:00.000071
Person Tagging: 0:00:00.000051
Person Tagging: 0:00:00.000109
Person Tagging: 0:00:00.000062
Person Tagging: 0:00:00.000133
Person Tagging: 0:00:00.000026
Person Tagging: 0:00:00.000155
Person Tagging: 0:00:00.000345
Person Tagging: 0:00:00.000153
Person Tagging: 0:00:00.000051
Person Tagging: 0:00:00.001023
Person Tagging: 0:00:00.000118
Person Tagging: 0:00:00.000048
Person Tagging: 0:00:00.000541
Person Tagging: 0:00:00.000026
Person Tagging: 0:00:00.000121
Person Tagging: 0:00:00.000492
Person Tagging: 0:00:00.000059
Person Tagging: 0:00:00.000132
Person Tagging: 0:00:00.000141
Person Tagging: 0:00:00.000093
Person T

Person Tagging: 0:00:00.000332
Person Tagging: 0:00:00.000247
Person Tagging: 0:00:00.000075
Person Tagging: 0:00:00.000100
Person Tagging: 0:00:00.000065
Person Tagging: 0:00:00.000070
Person Tagging: 0:00:00.000122
Person Tagging: 0:00:00.000035
Person Tagging: 0:00:00.000306
Person Tagging: 0:00:00.000238
Person Tagging: 0:00:00.000378
Person Tagging: 0:00:00.000287
Person Tagging: 0:00:00.000056
Person Tagging: 0:00:00.000178
Person Tagging: 0:00:00.000108
Person Tagging: 0:00:00.000365
Person Tagging: 0:00:00.000053
Person Tagging: 0:00:00.000095
Person Tagging: 0:00:00.000176
Person Tagging: 0:00:00.000274
Person Tagging: 0:00:00.000261
Person Tagging: 0:00:00.000259
Person Tagging: 0:00:00.000029
Person Tagging: 0:00:00.000109
Person Tagging: 0:00:00.000160
Person Tagging: 0:00:00.000099
Person Tagging: 0:00:00.000083
Person Tagging: 0:00:00.000094
Person Tagging: 0:00:00.000039
Person Tagging: 0:00:00.000046
Person Tagging: 0:00:00.000146
Person Tagging: 0:00:00.000313
Person T

Person Tagging: 0:00:00.000462
Person Tagging: 0:00:00.000266
Person Tagging: 0:00:00.000077
Person Tagging: 0:00:00.000282
Person Tagging: 0:00:00.001340
Person Tagging: 0:00:00.000037
Person Tagging: 0:00:00.000095
Person Tagging: 0:00:00.000283
Person Tagging: 0:00:00.000032
Person Tagging: 0:00:00.000035
Person Tagging: 0:00:00.000071
Person Tagging: 0:00:00.000130
Person Tagging: 0:00:00.000208
Person Tagging: 0:00:00.000286
Person Tagging: 0:00:00.000136
Person Tagging: 0:00:00.000118
Person Tagging: 0:00:00.000039
Person Tagging: 0:00:00.000222
Person Tagging: 0:00:00.000144
Person Tagging: 0:00:00.000503
Person Tagging: 0:00:00.000239
Person Tagging: 0:00:00.000055
Person Tagging: 0:00:00.000216
Person Tagging: 0:00:00.000307
Person Tagging: 0:00:00.000194
Person Tagging: 0:00:00.000202
Person Tagging: 0:00:00.000292
Person Tagging: 0:00:00.000213
Person Tagging: 0:00:00.000040
Person Tagging: 0:00:00.000079
Person Tagging: 0:00:00.000062
Person Tagging: 0:00:00.000106
Person T

Person Tagging: 0:00:00.001089
Person Tagging: 0:00:00.000070
Person Tagging: 0:00:00.000984
Person Tagging: 0:00:00.000273
Person Tagging: 0:00:00.000274
Person Tagging: 0:00:00.000281
Person Tagging: 0:00:00.000171
Person Tagging: 0:00:00.000074
Person Tagging: 0:00:00.001172
Person Tagging: 0:00:00.000101
Person Tagging: 0:00:00.000132
Person Tagging: 0:00:00.000498
Person Tagging: 0:00:00.000306
Person Tagging: 0:00:00.000598
Person Tagging: 0:00:00.000295
Person Tagging: 0:00:00.000237
Person Tagging: 0:00:00.000156
Person Tagging: 0:00:00.000618
Person Tagging: 0:00:00.000356
Person Tagging: 0:00:00.000036
Person Tagging: 0:00:00.000072
Person Tagging: 0:00:00.000239


In [9]:
temp = pd.concat([input_df, new], axis=1)

In [14]:
sum(temp["body"]!="")

18525

In [10]:
temp.to_csv("junk_clean_all.csv")

In [ ]:
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\n\\s*)+',value=' \n ')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\r\\s*)+',value=' \r ')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\t\\s*)+',value=' ')
input_df['Symptom'] = input_df['Symptom'].replace(regex='(\\s*\\r\\n\\s*)+',value=' \r\n ')

In [15]:
import re
from datetime import datetime
import itertools
def countTotal(ser,vecs):
    print(len(vecs))
    total = 0;
    for vec in vecs:
        a = ser.str.contains(vec).sum()
        print(a)
        #print(ser)
        #print(ser.str.contains("original"))
        total += a
    print("Total occurances - %d" %total)
    
def parse_text(text):
    for pattern, replacement in new_dict_comp.items():
        text = pattern.sub(replacement, text)
    return text

i = 50;
startTime1 = datetime.now()
temp["New_Symptom"] = temp["body"];
for i in range(300,9,-1):
    for j in range(10,9,-1):
        startTime = datetime.now()
        startTime2 = datetime.now()
        
        print(i , j)
        try:
            pattern = "[\\S]+";
            tf_vectorizer = CountVectorizer( min_df=j,
                                            ngram_range = (i,i),
                                            analyzer='word',
                                            tokenizer=lambda x: x.split(' '),
                                            lowercase = False
                                            )
            tf_vectorizer.fit(temp["New_Symptom"])
            print(tf_vectorizer.get_feature_names())
            print("total vector size - %d" %(len(tf_vectorizer.get_feature_names())))
            
            startTime3 = datetime.now()
            
            a =  zip(tf_vectorizer.get_feature_names(), itertools.repeat(""))
            new_dict = {re.escape(k): v for k, v in a}
            new_dict_comp = {re.compile(k): v for k, v in new_dict.items()}
#             countTotal(input_df["New_Symptom"],tf_vectorizer.get_feature_names())
            temp["New_Symptom"] = temp["New_Symptom"].apply(parse_text)
            
#             for vec in tf_vectorizer.get_feature_names():
#                 #print(input_df["New_Symptom"].str.contains(vec).sum())
#                 input_df["New_Symptom"] = input_df["New_Symptom"].str.replace(re.escape(vec),"")  

            print("time to replace - ")
            print(datetime.now() - startTime3)
            #countTotal(input_df["New_Symptom"],tf_vectorizer.get_feature_names())
            pattern = "[\\S]+";
            tf_vectorizer = CountVectorizer( min_df=j,
                                            ngram_range = (i,i),
                                            analyzer='word',
                                            tokenizer=lambda x: x.split(' '),
                                            lowercase = False
                                            )
            tf_vectorizer.fit(temp["New_Symptom"])
            print("total vector size after replacement - %d" %(len(tf_vectorizer.get_feature_names())))
            #print(tf_vectorizer.get_feature_names()[0])
            #input_df["New_Symptom"] = temp
            print("time to vectorize 2 times- ") 
            print(datetime.now() - startTime2)
        except ValueError:
            print(ValueError)
        except Exception as e:
            print(e)
        print(datetime.now() - startTime)
print("Total Time: - ")
print(datetime.now() - startTime1)

300 10
<class 'ValueError'>
0:00:01.074330
299 10
<class 'ValueError'>
0:00:00.943549
298 10
<class 'ValueError'>
0:00:00.936898
297 10
<class 'ValueError'>
0:00:00.945401
296 10
<class 'ValueError'>
0:00:00.939464
295 10
<class 'ValueError'>
0:00:00.945430
294 10
<class 'ValueError'>
0:00:00.920585
293 10
<class 'ValueError'>
0:00:00.929439
292 10
<class 'ValueError'>
0:00:00.991762
291 10
<class 'ValueError'>
0:00:01.022212
290 10
<class 'ValueError'>
0:00:01.007031
289 10
<class 'ValueError'>
0:00:00.988573
288 10
<class 'ValueError'>
0:00:00.946446
287 10
<class 'ValueError'>
0:00:00.940331
286 10
<class 'ValueError'>
0:00:00.936642
285 10
<class 'ValueError'>
0:00:00.931963
284 10
<class 'ValueError'>
0:00:00.969340
283 10
<class 'ValueError'>
0:00:01.012528
282 10
<class 'ValueError'>
0:00:00.998293
281 10
<class 'ValueError'>
0:00:00.970243
280 10
<class 'ValueError'>
0:00:01.053960
279 10
<class 'ValueError'>
0:00:01.057844
278 10
<class 'ValueError'>
0:00:01.065248
277 10
<cla

<class 'ValueError'>
0:00:03.520168
110 10
<class 'ValueError'>
0:00:01.697463
109 10
['network will be disabled until an IT Administrator unlocks your account.\nMobile Devices and Tablets, please note that you may also need to change your password on your mobile device once you have changed your password on the domain.\n   Network passwords are valid for a period of 90 days Password cannot be any of the 5 previously used passwords Passwords must contain 3 of the 4 types of characters    Uppercase (example: A) Lowercase (example: a) Number (example: 1) Symbol (example: $, @, etc.)   \nIf you have any questions or need further assistance, please contact the IT Service Desk by logging an incident through']
total vector size - 1
time to replace - 
0:00:00.030234
<class 'ValueError'>
0:00:03.434831
108 10
<class 'ValueError'>
0:00:01.702186
107 10
<class 'ValueError'>
0:00:01.756055
106 10
<class 'ValueError'>
0:00:01.772965
105 10
<class 'ValueError'>
0:00:01.759832
104 10
<class 'ValueEr

<class 'ValueError'>
0:00:04.441354
62 10
['Software (Lumension is now part of the HEAT Software group)\nThis email message is for the sole use of the intended recipient(s) and may contain proprietary, confidential or attorney-client privileged information. Any unauthorized review, use, disclosure or distribution is prohibited. If you are not the intended recipient, please contact the sender by reply email and destroy all copies of the original message. (AutoClosed)']
total vector size - 1
time to replace - 
0:00:00.028406
<class 'ValueError'>
0:00:04.494283
61 10
<class 'ValueError'>
0:00:02.297976
60 10
['Software (Lumension is now part of the HEAT Software group)\nThis email message is for the sole use of the intended recipient(s) and may contain proprietary, confidential or attorney-client privileged information. Any unauthorized review, use, disclosure or distribution is prohibited. If you are not the intended recipient, please contact the sender by reply email and destroy all cop

<class 'ValueError'>
0:00:04.873674
46 10
['This message is intended for the addressee(s) only and may contain confidential or privileged \ninformation. Any use of this information by persons other than addressee(s) is prohibited. If you \nhave received this message in error, please reply to the sender and delete or destroy all copies.', 'message is intended for the addressee(s) only and may contain confidential or privileged information. Any use of this information by persons other than addressee(s) is prohibited. If you have received this message in error, please reply to the sender and delete or destroy all copies (AutoClosed)']
total vector size - 2
time to replace - 
0:00:00.042595
<class 'ValueError'>
0:00:04.825531
45 10
['This message is intended for the addressee(s) only and may contain confidential or privileged information.\nAny use of this information by persons other than addressee(s) is prohibited.\nIf you have received this message in error, please reply to the sender an

<class 'ValueError'>
0:00:05.915757
27 10
['The server has tried to deliver this message, without success, and has stopped trying. Please try sending this message again. If the problem continues, contact your helpdesk.', 'ensure their first day is as seamless as possible please provide us with the following information no later than 10 days prior to their start date..\n ']
total vector size - 2
time to replace - 
0:00:00.044445
<class 'ValueError'>
0:00:06.200345
26 10
['\nSet up team: Please ensure you have set up the contingent worker in the\nFor any questions related to set up or term, the Manager should be', 'Set up team: Please ensure you have set up the contingent worker in the\nFor any questions related to set up or term, the Manager should be', 'Set up team: Please ensure you have set up the employee in the assigned\nFor any questions related to set up or term, the Manager should be', 'with the\nSet up team: Please ensure you have enabled access for the Contractor in\nFor any q

<class 'ValueError'>
0:00:07.663224
10 10
[': new support telephone number for the UK : +44', 'Software (Lumension is now part of the HEAT Software group)', 'Software( Lumension is now part of the HEAT Software group)', 'Support Renewal Team Lead – USA, Canada, and Latin America', 'Thanks,  Jerrica Owens-Reeder, Sales Operations Analyst Heat Software |', 'are currently no logon servers available to service the logon', 'privileged information. Any unauthorized review, use, disclosure or distribution is', 'received a request to reset the password for your NetSuite', 'to manage non-windows and 3rd party patching in Microsoft System', 'unexpected error has occurred. Your development organization has been notified.', 'which email distribution lists will they need to be added?']
total vector size - 11
time to replace - 
0:00:00.161652
<class 'ValueError'>
0:00:08.404934
Total Time: - 
0:10:22.390511


In [16]:
temp.to_csv("junk_clean_repeated_chunks_removed_all.csv")

In [ ]:
input_df = pd.readcsv()

In [3]:
for i in range(1,20):
    print(repr(input_df['Symptom'][i]))
    print("\n-------------\n")

'Whenever I send an email to Brendan.kelley@heatsoftware.com from Outlook, I get the following error. I am able to send him emails from my phone without any issues. I deleted his saved email address from Outlook but I still have issues. I need to get this fixed asap please.   Thanks, Brian     Brian Shepard Information Technology – Enterprise Apps HEAT Software 425.210.2637 direct Brian.Shepard@heatsoftware.com | www.HEATsoftware.com        _____________________________________________\r\n From: Microsoft Outlook \r\nSent: Sunday, June 26, 2016 3:31 PM\r\n To: Brian Shepard\r\n Subject: Undeliverable: FW: test     Delivery has failed to these recipients or groups:   Brendan Kelley\r\n The email address you entered couldn\'t be found. Please check the recipient\'s email address and try to resend the message. If the problem continues, please contact your helpdesk.\r\n \r\n                Diagnostic information for administrators:   Generating server: cs-mbx01v.corp.lcl\r\n    IMCEAEX-_O=

In [5]:
count = 0
index = []
for i in range(input_df.shape[0]):
    if len(input_df['Symptom'][i]) > 3000:
        count+=1
        index.append(i)
#         print(input_df['Symptom'][i])
#         print("\n-------------\n")

print(count)
print(index)                     

338
[8, 59, 147, 186, 220, 263, 275, 329, 336, 346, 397, 467, 494, 496, 532, 534, 538, 691, 725, 791, 800, 840, 843, 876, 894, 911, 955, 966, 987, 1123, 1163, 1186, 1199, 1247, 1281, 1339, 1346, 1439, 1489, 1509, 1510, 1515, 1566, 1595, 1611, 1631, 1757, 1759, 1778, 1806, 1814, 1824, 1868, 1869, 1879, 1881, 1884, 1886, 1889, 2037, 2058, 2060, 2114, 2120, 2147, 2285, 2402, 2403, 2406, 2425, 2442, 2468, 2473, 2480, 2579, 2591, 2609, 2621, 2676, 2720, 2736, 2746, 2814, 2824, 2856, 2871, 2873, 3051, 3202, 3309, 3429, 3491, 3492, 3629, 3710, 3800, 3840, 3876, 3976, 4000, 4005, 4022, 4023, 4079, 4168, 4241, 4242, 4397, 4459, 4461, 4487, 4513, 4550, 4551, 4562, 4563, 4651, 4713, 4744, 4755, 4769, 4783, 4788, 4813, 4895, 4918, 4939, 4946, 5008, 5059, 5082, 5131, 5159, 5167, 5222, 5249, 5282, 5296, 5303, 5327, 5375, 5385, 5402, 5461, 5571, 5656, 5666, 5728, 5745, 5829, 5838, 5866, 5867, 5875, 5897, 5901, 5961, 6063, 6095, 6098, 6154, 6166, 6222, 6249, 6251, 6262, 6265, 6319, 6328, 6402, 6430, 6

In [12]:
# for i in range(20):
#     print(re.findall("\\sDiagnostic information for administrators\\s", input_df['Symptom'][i]))
# #     print("\n" + input_df['Subject'][i])
#     print("\n-------------\n")

count = 0
index =[]
for j in range(input_df.shape[0]):
    if len(re.findall("><", input_df['Symptom'][j])) > 0:
        index.append(j)
        count+=1
        
print(count)
print(index)

133
[59, 263, 327, 346, 397, 691, 843, 876, 894, 1186, 1199, 1247, 1308, 1699, 2010, 2120, 2356, 2402, 2403, 2426, 2435, 2480, 2591, 2736, 2845, 2871, 2887, 3202, 3491, 3492, 3976, 4079, 4168, 4400, 4513, 4617, 4639, 4769, 4788, 4813, 4895, 5082, 5167, 5296, 5303, 5314, 5385, 5402, 5483, 5641, 5901, 5938, 6098, 6154, 6262, 6319, 6474, 6500, 6508, 6598, 6629, 6741, 6793, 7152, 7182, 7204, 7321, 7355, 7709, 7789, 7910, 7967, 8030, 8098, 8108, 8367, 8441, 8496, 8515, 8521, 8562, 8596, 8628, 8630, 8655, 8656, 8661, 8684, 8718, 8753, 8799, 8814, 8837, 8842, 8855, 8899, 8902, 8971, 8985, 9019, 9023, 9162, 9200, 9223, 9233, 9438, 9687, 9743, 9771, 9974, 10007, 10022, 10047, 10052, 10056, 10071, 10120, 10168, 10336, 10341, 10374, 10378, 10489, 10572, 10609, 10806, 10824, 10868, 11026, 11134, 11187, 11208, 11225]


In [103]:
message = """Good morning,
 
 
 
Can you please contact me as soon as possible regarding this issue?
 
Attached is the screen Michal gets when he boots his PC
 
He is just back from holiday and nothing was done to the machine as far as we can tell
 
 
 
Thank you for your help
 
Benjamin Elzingre

Technical Support Manager – EMEA
 
Ivanti

benjamin.elzingre@ivanti.com | www.heatsoftware.com | www.ivanti.com (AutoClosed)
 """

In [14]:
import talon
talon.init()
from talon import signature
from talon.signature.bruteforce import extract_signature
message = input_df['Symptom'][4]
text_b, signature_b = extract_signature(message)
text, signature = signature.extract(message,sender="")
print(signature_b)
print("\n-----------\n")
print(signature)

None

-----------

None


In [15]:
print(message)

Is this real?
 
 
 
Ken
 
 
 
 
 
From: Ivanti Security Team [mailto:mike.lloyd@ivanti.com] 
Sent: Sunday, April 2, 2017 9:02 PM
 To: Kenneth Folsom <Kenneth.Folsom@ivanti.com>
 Subject: You have been assigned Security Awareness Training
 
 
  
 
 
Hi Kenneth,
 
The Ivanti Security team is launching a new Password Training module for our security training initiative. We will be sharing educational tools that can assess and improve your security intelligence both at work and at home. You have an assignment awaiting your completion. You should be able to complete this training in approximately 10 - 20 minutes. Please click the link below to start your assignment, which is due by 04/28/2017.
 
https://ivanti.securityeducation.com/ticketAuth/fe059c2eb93e4fb2a396770116680bda
 
If you would like, you may review any of the other training modules available.
 
If you have any questions or any feedback, please reach out to me at mike.lloyd@ivanti.com.
 
Thank you,
 Mike Lloyd
 
 
 
P.S. The URL 

In [10]:
text

'Hello,\r\n \r\n \r\n \r\nIPCM ITSM down system pager number 719-278-7264 not routing to on call pager.\r\n \r\n \r\n \r\nRegards,\r\n \r\n \r\n \r\n    \r\n\r\n         \r\nGlenn Secrest\r\n \r\nTechnical Analyst 2, \r\n\r\nGlobal Support Services\r\n \r\n     \r\n   \r\no +1 719 278.7273\r\n \r\nw  www.ivanti.com\r\n   \r\n   \r\n \r\n \r\n    \r\n\r\n \r\n     \r\n   \r\nNote: My new email address is: Glenn.Secrest@ivanti.com\r\n   \r\n   \r\n \r\n \r\n8415 Explorer Drive, Suite 150 Colorado Springs, CO 80920\r\n \r\n \r\n \r\nThis message is intended for the addressee(s) only and may contain confidential or privileged information. \r\n\r\nAny use of this information by persons other than addressee(s) is prohibited. \r\n\r\nIf you have received this message in error, please reply to the sender and delete or destroy all copies. (AutoClosed)'

In [11]:
signature

In [40]:
signature_b

In [13]:
from nltk import word_tokenize, pos_tag, ne_chunk
tagged = ne_chunk(pos_tag(word_tokenize(input_df['Symptom'][4])))
person = []
for subtree in tagged.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
            
print(person)

['Ivanti', 'Security', 'Team', 'Kenneth', 'Folsom', 'Kenneth', 'Please', 'Mike', 'Lloyd']


In [61]:
b = "\\[a-d\\]fr"
a = pd.Series(["dfr ghtf [a-d]fr", "drf\| drf sbghj, kjdjfds a-dfr", "dfr nkskdjfn"])
a = a.str.replace(b,"")
a

0                         dfr ghtf 
1    drf\| drf sbghj, kjdjfds a-dfr
2                      dfr nkskdjfn
dtype: object

In [62]:
b = "[a-d]fr"
a = pd.Series(["dfr ghtf [a-d]fr", "drf\| drf sbghj, kjdjfds a-dfr", "dfr nkskdjfn"])
a = a.str.replace(b,"")
a

0                   ghtf [a-d]fr
1    drf\| drf sbghj, kjdjfds a-
2                       nkskdjfn
dtype: object

In [63]:
b = "\[a-d\]fr"
a = pd.Series(["dfr ghtf [a-d]fr", "drf\| drf sbghj, kjdjfds a-dfr", "dfr nkskdjfn"])
a = a.str.replace(b,"")
a

0                         dfr ghtf 
1    drf\| drf sbghj, kjdjfds a-dfr
2                      dfr nkskdjfn
dtype: object